# UI

> A chat interface UI demo

In [ ]:
#| export
from fasthtml.common import *
from fhbasecoat.utils import *
from fhbasecoat.common import *
from fhbasecoat.interactive import *
from fhbasecoat.chat import *
import fasthtml.components as fh
from fasthtml.jupyter import *
from fastcore.meta import delegates
from fastcore.utils import *
from itertools import product
from enum import Enum, auto
import nbformat
import itertools
from functools import wraps

In [ ]:
code_highlight_hdrs = (
    Link(rel="stylesheet", 
             href="https://cdnjs.cloudflare.com/ajax/libs/highlight.js/11.9.0/styles/atom-one-light.min.css",
             id="hljs-light"),
    Link(rel="stylesheet", 
             href="https://cdnjs.cloudflare.com/ajax/libs/highlight.js/11.9.0/styles/atom-one-dark.min.css",
             id="hljs-dark"),
    Script(src="https://cdnjs.cloudflare.com/ajax/libs/highlight.js/11.9.0/highlight.min.js"))

# preload_hdrs = (Script(src="https://unpkg.com/htmx-ext-preload@2.1.0/preload.js"),)

# custom_hdrs = code_highlight_hdrs + preload_hdrs

In [ ]:
# icons = [
#     "chevron-down", "chevron-right", "chevrons-up-down", "search", "x",
#     "plus", "arrow-up", "panel-left", "loader-circle", "sun", "moon",
#     "circle-check", "chevron-left", "ellipsis"
# ]

In [ ]:
FastHTML??


```python
@delegates(ori_FastHTML, keep=True, but=["pico"])
def FastHTML(hdrs=None, ftrs=None, pico=False, icons=[], code_highlight=True, **kwargs):
    hdrs = basecoat_hdrs + (hdrs or ())
    if code_highlight:
        hdrs += (
    Link(rel="stylesheet", href="https://cdnjs.cloudflare.com/ajax/libs/highlight.js/11.9.0/styles/atom-one-light.min.css", id="hljs-light"),
    Link(rel="stylesheet", href="https://cdnjs.cloudflare.com/ajax/libs/highlight.js/11.9.0/styles/atom-one-dark.min.css", id="hljs-dark"),
    Script(src="https://cdnjs.cloudflare.com/ajax/libs/highlight.js/11.9.0/highlight.min.js"))
    spritesheet.nms.update(icons)
    ftrs = (ftrs or ()) + (spritesheet,)  # Add sprite sheet to footers
    return ori_FastHTML(hdrs=hdrs, ftrs=ftrs, pico=pico, **kwargs)
```

**File:** `~/git/fhbasecoat/fhbasecoat/utils.py`

In [ ]:
CodeHighlightThemeScript??


```python
def CodeHighlightThemeScript(
    custom_kws:str # A space separated string of variable names to highlight
):
    return (
        Script(f"""
            function updateHljsTheme() {{
                const isDark = document.documentElement.classList.contains('dark');
                document.getElementById('hljs-light').disabled = isDark;
                document.getElementById('hljs-dark').disabled = !isDark;
            }}
            updateHljsTheme();

            hljs.registerLanguage('python-custom', function(hljs) {{
                var python = hljs.getLanguage('python');
                python.keywords.built_in += ' {custom_kws}';
                return python;
            }});

            document.addEventListener('basecoat:theme', () => setTimeout(updateHljsTheme, 0));
            hljs.highlightAll();
        """),
    )
```

**File:** `~/git/fhbasecoat/fhbasecoat/chat.py`

In [ ]:
def CodeHighlightThemeScript(
    custom_kws:str # A space separated string of variable names to highlight
):
    return (
        Script(f"""
            function updateHljsTheme() {{
                const isDark = document.documentElement.classList.contains('dark');
                document.getElementById('hljs-light').disabled = isDark;
                document.getElementById('hljs-dark').disabled = !isDark;
            }}
            updateHljsTheme();

            hljs.registerLanguage('python-custom', function(hljs) {{
                var python = hljs.getLanguage('python');
                python.keywords.built_in += ' {custom_kws}';
                return python;
            }});

            document.addEventListener('basecoat:theme', () => setTimeout(updateHljsTheme, 0));
            document.body.addEventListener('htmx:afterSettle', () => {{
                hljs.highlightAll();
            }});
            hljs.highlightAll();
        """),
    )

In [ ]:
custom_highlighting = " ".join([name for name in globals() if name[0].isupper()])
hdrs = CodeHighlightThemeScript(custom_highlighting)

In [ ]:
app = FastHTML(ftrs=hdrs, exts=("ws", "preload"), session_cookie="mysession")
rt = app.route

For some reason this code highlightin gis not working. Should I be loading it in another spot?
```
VM231 alert:4 Uncaught TypeError: Cannot set properties of null (setting 'disabled')
    at updateHljsTheme (VM231 alert:4:64)
    at VM231 alert:7:13
```

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠

The script is running before the stylesheet `<link>` elements are in the DOM. The `CodeHighlightThemeScript` is in `hdrs`, but it executes before the page fully loads.

Quick fix: wrap the initialization in `DOMContentLoaded`:

```javascript
document.addEventListener('DOMContentLoaded', () => {
    updateHljsTheme();
    hljs.highlightAll();
});
```

Or alternatively, move `CodeHighlightThemeScript` to `ftrs` instead of `hdrs` so it runs after the body loads.

Which approach would you prefer to try?

Okay, it is better with ftrs. But now every time I go to a new page/link my highlighting stops. Note that I'm using hx_boost.

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠

With `hx-boost`, HTMX swaps the body content but doesn't re-execute footer scripts. You need to re-run `hljs.highlightAll()` after each swap.

Add this to your script:

```javascript
document.body.addEventListener('htmx:afterSettle', () => {
    hljs.highlightAll();
});
```

This will trigger highlighting after each HTMX content swap.

In [ ]:
@app.middleware("http")
async def add_cache_headers(request, call_next):
    response = await call_next(request)
    if "/documentation-rag/" not in str(request.url):
        response.headers["Cache-Control"] = "max-age=300"  # 5 minutes
    return response

In [ ]:
# srv.stop()

In [ ]:
srv = JupyUvi(app)

In [ ]:
from pathlib import Path

In [ ]:
! ls ~/git/fhbasecoat/fhbasecoat

__init__.py  _modidx.py  common.py	 utils.py
__pycache__  chat.py	 interactive.py


In [ ]:
common_components = Path("~/git/fhbasecoat/fhbasecoat/common.py").expanduser().read_text()
interactive_components = Path("~/git/fhbasecoat/fhbasecoat/interactive.py").expanduser().read_text()

Note: whenever you answer questions from now you should refer to $`[common_components, interactive_components]`.

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

Got it! I can see you're working with FastHTML and have loaded some component code from `fhbasecoat`. I'll reference the `common_components` and `interactive_components` code when answering your questions.

What would you like to explore or work on with these components?

In [ ]:
slug_map = {}

In [ ]:
def make_key():
    current = [None]
    def key_fn(cell):
        if cell.cell_type == "markdown" and cell.source.startswith("###"):
            current[0] = cell.source.strip("###").strip()
        return current[0]
    return key_fn

In [ ]:
def mk_nbdict(filename:str):
    nb = nbformat.read(filename, as_version=4)
    cells = itertools.islice(itertools.dropwhile(lambda c: c.source != "# Components", nb.cells), 1, None)
    nbdict = {}
    for header, group in itertools.groupby(cells, key=make_key()):
        header_slug = slugify(header)
        slug_map[header_slug] = header
        nbdict[header_slug] = list(group)
    return nbdict 


In [ ]:
matched = re.search(r"\d+_(\w+)\.ipynb", "01_common.ipynb").group(1)
matched

'common'

In [ ]:
def format_nm(nm:str):
    return re.search(r"\d+_(\w+)\.ipynb", nm).group(1).title()

In [ ]:
documentation = {format_nm(nm): mk_nbdict(nm) for nm in ["01_common.ipynb", "02_interactive.ipynb", "03_chat.ipynb"]}

In [ ]:
documentation.keys()

dict_keys(['Common', 'Interactive', 'Chat'])

# Code

### Site components

In [ ]:
def SearchBarButton(id):
    return DialogOpenButton(
        Div(
            Div(
                Icon("search", cls="size-4"),
                Span("Search Documentation", cls="text-sm"),
                cls="flex gap-2 items-center",
            ),
            Kbd("⌘ K", cls="text-2xs"),
            cls="flex items-center px-3 py-2 justify-between",
        ),
        cls="m-1 hover:cursor-pointer bg-background rounded-lg w-full",
        did=id,
    )

In [ ]:
def SearchBarCommand(id):
    contents = []
    for category, nbdict in documentation.items():
        command_items = [
            A(CommandItem(slug_map[nm]), 
            href=f"/{category.lower()}/{nm}",
            cls="block w-full",
            onclick="this.closest('dialog').close()")
            for nm in nbdict.keys()
        ]
        contents.append(
            CommandGroup(
                ItemHeader(category),
                *command_items,
            )
        )
        contents.append(Separator())
    contents.pop()
    
    return CommandDialog(
        Command(
            CommandSearch(),
            CommandScrollable(
                *contents,
                style="height: 400px;",
            ),
            cls="w-4xl",
            style="top: 29%; max-width: 650px; max-height: 500px;",

        ),
        id=id,


    )

In [ ]:
def SearchBar(id="search-bar"):
    return Div(
        SearchBarButton(id=id),
        SearchBarCommand(id=id),
        Script(f"""
        document.addEventListener('keydown', (e) => {{
            if ((e.ctrlKey || e.metaKey) && e.key === 'k') {{
                e.preventDefault();
                const dialog = document.getElementById("{id}");
                if (dialog.open) {{
                    dialog.close();
                }} else {{
                    dialog.showModal();
                    dialog.querySelector("header input")?.focus();
                }}
            }}
        }});
        """),
        cls="flex w-full",
    )

In [ ]:
category_icons = {"Common": None, "Interactive": None, "Chat": None}

In [ ]:
category_icons = {"Common": "blocks", "Interactive": "mouse-pointer-click", "Chat": "message-square"}

In [ ]:
for category, nbdict in documentation.items():
    print(category, nbdict.keys())

Common dict_keys(['accordion', 'alert', 'avatar', 'badge', 'breadcrumb', 'button', 'button-group', 'card', 'checkbox', 'form', 'kbd', 'labels-inputs', 'pagination', 'progress', 'radio-group', 'skeleton', 'slider', 'spinner', 'switch', 'table', 'textarea', 'theme-switcher', 'toast', 'tooltip'])
Interactive dict_keys(['dialog-alert-dialog', 'dropdown-menu', 'popover', 'select-combobox', 'tabs', 'sidebar', 'command'])
Chat dict_keys(['chat-input', 'chat-message', 'chat-code'])


In [ ]:
list(map(lambda k: slug_map[k], nbdict.keys()))

['Chat Input', 'Chat Message', 'Chat code']

In [ ]:
def DemoSidebar():
    contents = [
        SidebarCollapsable(
            IconTitle(category, icon=category_icons.get(category)), 
            list(map(lambda k: slug_map[k], nbdict.keys())),
            href_list=nbdict.keys(),
            href_parent=category,
        ) 
        for category, nbdict in documentation.items()
    ]
    return Sidebar(
        SearchBar(),
        Group(
            "Components", 
            *contents
        ),
        SidebarGroup("Demos", ["Documentation RAG"], icon_list=["bot"], href_parent="demos"),
        footer=Div(
                Dropdown(
                    DropdownItem("Upgrade to Pro"),
                    Separator(),
                    DropdownItem("Account"),
                    trigger_btn=DropdownTriggerButton(
                        Div(
                            Avatar(src="https://ui.shadcn.com/avatars/shadcn.jpg"), Span("Taya", cls="pl-2"),
                            cls="flex items-center w-full",
                        ),
                        Icon("chevrons-up-down"), did="sidebar-dropdown", 
                        cls=f"{ButtonT.ghost} justify-between py-6 px-3 m-1",
                        style="width: calc(100% - 0.5rem)"
                    ),
                    id="sidebar-dropdown",
                    side="right", align="end",
            ),
            cls="flex flex-col items-left w-full",
        ),
        hx_boost="true", hx_target="#main-content", hx_select="#main-content", hx_ext="preload", preload="mouseover",
    )

# pw(
#     DemoSidebar()
# )

In [ ]:
def TopBar(category="", component=""):
    return Div(
        Div(
            ToggleButton(),
            SeparatorVertical(),
            Breadcrumb([category.title(), slug_map.get(component, component.title())]),
            cls="flex items-center gap-2"
        ),
        ThemeSwitcher(),
        cls="flex justify-between items-center w-full px-3 overflow-hidden py-1 border-b"
    )

# pw(TopBar("Chats", "Demo Chat"))

### Chat interface

In [ ]:
from lisette import *
chat = Chat(model="anthropic/claude-3-haiku-20240307")
chat

In [ ]:
# for r in chat("Testing2", stream=True):
#     delta = r.choices[0].delta if hasattr(r.choices[0], 'delta') else None
#     if delta and delta.content:
#         print(delta.content, end='', flush=True)
# final_response = r.choices[0].message.content

In [ ]:
@app.ws("/send_msg_ws")
async def send_msg_ws(text:str, send):
    await send(Div(ChatPrompt(text), id="chat-interface", hx_swap_oob="beforeend"))
    r = chat(text, stream=False)
    output = r.choices[0].message.content
    await send(Div(ChatAssistant(output), id="chat-interface", hx_swap_oob="beforeend"))

In [ ]:
def sidebar_layout(fn):
    @wraps(fn)
    def inner_fn(*args, **kwargs):
        return Div(
            DemoSidebar(),
            Main(
                TopBar(**kwargs),
                Div(
                    fn(*args, **kwargs),
                    cls="flex flex-col flex-1 overflow-y-auto min-h-0",
                ),
                cls="h-full flex flex-col",
                id="main-content"
            ),
            cls="h-screen w-full",
        )
    return inner_fn

In [ ]:
def DemoChat():
    return ChatInterface(
            ChatPrompt("A test message sent by a user\n\nAnd new lines\nAre here."),
            ChatAssistant("A smaller response returned by a LLM. Great question. You are absolutely right!"),
            chat_input=ChatInput(
                cls="mb-1 mt-0", ta_cls="rounded-t-none", width="w-full",
                hx_ext="ws", ws_connect="/send_msg_ws", ws_send=True,
            ),
            cls="flex-1",
            id="chat-interface",
        )

In [ ]:
@rt("/demos/documentation-rag")
@sidebar_layout
def demo_chat2():
    return Div(
        DemoChat(),
        cls="flex flex-1",
    )

TIL: css width flows down from parent to child, but height doesn't. h-full only works if parent has an explicit height. I think flex-1 does as well. So you need to go down the flow of things to figure out the height at each level.
- Explicit height only by h-screen, h-[num], flex-1 (if parent).

By default h of elements is to be as small as possible.

For width element it's to take up 100% of parent width.

### Dyanmic routes

In [ ]:
nbdict = documentation["Common"]
len(nbdict)

24

In [ ]:
component = nbdict["breadcrumb"]
len(component)

6

In [ ]:
def Tabs(contents:list, tablist:list, id:str, default_tab=0, orientation="horizontal", cls="w-96", nav_cls="w-fit"):
    nav_items = [TabNav(o, id, idx, selected=(idx==default_tab)) for idx, o in enumerate(tablist)]
    for idx, content in enumerate(contents):
        active_dict = {"aria_selected": "true"} if idx==default_tab else {"aria_selected": "false", "hidden": True}
        content.attrs.update({"role": "tabpanel", "id": f"{id}-panel-{idx}", "aria_labelledby": f"{id}-tab-{idx}", "tabindex": "-1", **active_dict})

    return Div(
        Nav(
            *nav_items,
             role="tablist", aria_orientation=orientation, cls=nav_cls,
        ),
        *contents,
        cls=f"tabs {cls}", id=id,
    )

In [ ]:
def DemoCard(nm, fn, src):
    return Div(
        # CodeHighlighting(),
        Tabs(
            contents=[
                CardOutline(fn(), cls="rounded-lg min-h-96 flex justify-center items-center py-5"),
                Card(
                    Pre(
                        Code(src, cls="language-python-custom", style="background: transparent; overflow: visible;"),
                    ), 
                cls="rounded-lg min-h-96 flex text-sm overflow-auto overflow-y-auto",
                ),
            ],
            tablist=["preview", "code"],
            id=f"{nm}-tab",
            cls="w-full",
            nav_cls="w-fit ml-auto",
        ),
        cls="w-full",
    )

Next steps for democard:
- Add page title and democard title
- Ensure multiple demo cards are supported
- Make tabs smaller on the side
- Add copy to clipboard button for code
- Add rest of source code accordinon toggle

Steps for render demos:
1. Find all demo cells
2. Excec them to get into ns
3. Render ns + demo code onto page

In [ ]:
def render_demos(component):
    ns = {}
    demo_cells = L(component).filter(lambda c: c.source.startswith("#| demo"))
    demos = []
    for cell in demo_cells:
        exec(cell.source, globals(), ns)
        name = list(ns.keys())[-1]
        demos.append((name, ns[name], cell.source))
    return [DemoCard(*d) for d in demos]

In [ ]:
pw(*render_demos(component))

In [ ]:
component[0].source.strip("#").strip()

'Breadcrumb'

In [ ]:
src_list = L(component).filter(lambda c: not c.source.startswith(("#| demo", "#| preview")))[1:]
src_list

(#3) [{'cell_type': 'code', 'execution_count': None, 'id': '7a643172', 'metadata': {}, 'outputs': [], 'source': '#| export\ndef accumulate_hrefs(links:list):\r\n    slugs = [slugify(o) for o in links]\r\n    return list(accumulate(slugs, lambda a, b: f"{a}/{b}", initial=""))[1:]'},{'cell_type': 'code', 'execution_count': None, 'id': 'f7e8eff5', 'metadata': {}, 'outputs': [{'data': {'text/plain': "['/home',\n '/home/documents',\n '/home/documents/components',\n '/home/documents/components/breadcrumb']"}, 'execution_count': None, 'metadata': {}, 'output_type': 'execute_result'}], 'source': 'accumulate_hrefs(["Home", "Documents", "Components", "Breadcrumb"])'},{'cell_type': 'code', 'execution_count': None, 'id': 'c17476fb', 'metadata': {}, 'outputs': [], 'source': '#| export\ndef Breadcrumb(links:list, hrefs=None):\n    if not hrefs: hrefs = accumulate_hrefs(links)\n    contents = []\n    for link, href in zip(links[:-1], hrefs):\n        contents.append(Li(A(link, href=href, cls="hover:t

In [ ]:
src_list[0]

{'cell_type': 'code',
 'execution_count': None,
 'id': '7a643172',
 'metadata': {},
 'outputs': [],
 'source': '#| export\ndef accumulate_hrefs(links:list):\r\n    slugs = [slugify(o) for o in links]\r\n    return list(accumulate(slugs, lambda a, b: f"{a}/{b}", initial=""))[1:]'}

In [ ]:
src_list[1]

{'cell_type': 'code',
 'execution_count': None,
 'id': 'f7e8eff5',
 'metadata': {},
 'outputs': [{'data': {'text/plain': "['/home',\n '/home/documents',\n '/home/documents/components',\n '/home/documents/components/breadcrumb']"},
   'execution_count': None,
   'metadata': {},
   'output_type': 'execute_result'}],
 'source': 'accumulate_hrefs(["Home", "Documents", "Components", "Breadcrumb"])'}

In [ ]:
src_list[1].outputs[0]["data"]["text/plain"]

"['/home',\n '/home/documents',\n '/home/documents/components',\n '/home/documents/components/breadcrumb']"

In [ ]:
def strip_meta(text, prefixes=["#| exports", "#| export", "\n"]):
    for prefix in prefixes:
        text = text.removeprefix(prefix)
    return text

In [ ]:
def get_output_text(output):
    if isinstance(output, dict):
        if "data" in output:
            data = output["data"]
            return data.get("text/plain") or data.get("text/html") or str(data)
        elif "text" in output:
            return output["text"]
    return str(output)

In [ ]:
def SourceCodeMsg(src_dict:dict):
    extra = []
    outputs = src_dict.outputs
    for output in outputs:
        text = get_output_text(output)
        if text:
            extra.append(ChatOutput(strip_meta(text)))     

    return Div(
        ChatCode(strip_meta(src_dict.source)),
        *extra,
        cls="flex flex-col gap-2"
    )

In [ ]:
def render_code_or_md(src_dict:dict):
    if src_dict["cell_type"] == "markdown":
        return ChatNote(src_dict.source)
    elif src_dict["cell_type"] == "code":
        return SourceCodeMsg(src_dict)
    else:
        raise ValueError("Invalid cell type")

In [ ]:
def render_sourcecode(component_list:list):
    src_list = L(component_list).filter(lambda c: not c.source.startswith(("#| demo", "#| preview")))[1:]
    contents = src_list.map(render_code_or_md)
    return Accordion(
        AccordionItem(
            "Source code",
            ChatInterface(
                *contents,
                inner_cls="border rounded-lg",
            )
        ),
        cls="mt-4"
    )


In [ ]:
@rt("/{category}/{component}")
@sidebar_layout
def get(category: str, component: str):
    try:
        nbdict = documentation[category.title()]
        component_dict = nbdict[component]
        return Div(
            H3(slug_map[component], cls="text-xl font-semibold tracking-tight mt-3"),
            Div(*render_demos(component_dict), cls="flex flex-col gap-3"),
            render_sourcecode(component_dict),
            cls="flex flex-col w-full lg:max-w-4xl mx-auto",
        )
    except Exception as e:
        return Div("ERROR", e)

In [ ]:
# This route needs to be defined in the demo for the toast example to work
@rt
def toast_example():
    return Toast("Success", "Toast send from backend!")

## Things to add

- A llm backend integration that puts out prompt and assistant
- Backend websockets for streaming
- Token count in top panel using tiktoken (And token (i) hover showing cost)
- Click to edit prompt/assistant output
- LLM thinking mode button + display live thinking output
- Switch between prompt + Note
- Note Output
- Note output integration with 
- Python result using web-python interpreter
- Code output type
- Hotkeys for sidebar, submit, switch type, etc.

### Things I've learnt

When using flexboxes, a child has to have a parent with flex to be able to use it.
For dynamic styling you can do `w-full lg:max-w-4xl mx-auto` to have something dynamically resize from full to a larger size.

htmx things:
- hotkeys by doing `hx_trigger="keydown[ctrlKey && key=='Enter'] from:body`
- Reset form by doing `hx_on__after_request="this.reset()"`
- Button auto get `type="submit"` when inside a form.
- To allow trigger to work with clicks as well you can do `click from:button, keydown...`
- To disable during the request you can do `hx_disabled_elt="find button, find textarea"`
- To clear textarea you can do `hx_on__before_request="this.reset()"`